# Intro to MLOps using ZenML

## 🌍 Overview

This repository is a minimalistic MLOps project intended as a starting point to learn how to put ML workflows in production. It features: 

Follow along this notebook to understand how you can use ZenML to productionalize your ML workflows!

<img src="_assets/pipeline_overview.png" width="50%" alt="Pipelines Overview">

# ⌚ Step 1: (Feature engineering) + Training pipeline

Lets run the feature engineering pipeline

<img src="_assets/feature_engineering_pipeline.png" width="50%" alt="Training pipeline">

In [ ]:
!python run.py --feature-pipeline

Initiating a new run for the pipeline: feature_engineering.
Reusing registered version: (version: 28).
New model version 4 was created.
Executing a new run.
Using user: hamza@zenml.io
Using stack: local-sagemaker-step-operator-stack
  model_registry: mlflow
  step_operator: sagemaker-eu
  experiment_tracker: mlflow
  container_registry: aws-eu
  orchestrator: default
  image_builder: local
  artifact_store: s3-zenfiles
Could not import GCP service connector: No module named 'google.api_core'.
Could not import Azure service connector: No module named 'azure.identity'.
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 8.117s.
Step data_splitter has started.
Step data_splitter has finished in 12.379s.
Step data_preprocessor has started.


Lets run the training pipeline

<img src="_assets/training_pipeline.png" width="50%" alt="Training pipeline">

In [ ]:
!python run.py --training-pipeline

# 🫅 Step 2: The inference pipeline

The batch inference pipeline simply takes the model marked as `production` and runs inference on it
with `live data`. The critical step here is the `inference_predict` step, where we load the model in memory
and generate predictions:

<img src="_assets/inference_pipeline.png" width="45%" alt="Inference pipeline">

In [ ]:
!python run.py --inference-pipeline

## Step 3: Deploying the pipeline to Huggingface

<img src="_assets/deployment_pipeline.png" width="45%" alt="Deployment pipeline">

In [ ]:
!python run.py --deployment-pipeline

## Congratulations!

You're a legit MLOps engineer now! You trained two models, evaluated them against
a test set, registered the best one with the ZenML model control plane,
and served some predictions. You also learned how to iterate on your models and
data by using some of the ZenML utility abstractions. You saw how to view your
artifacts and models via the client as well as the ZenML Dashboard.

## Further exploration

This was just the tip of the iceberg of what ZenML can do; check out the [**docs**](https://docs.zenml.io/) to learn more
about the capabilities of ZenML. For example, you might want to:

- [Deploy ZenML](https://docs.zenml.io/user-guide/production-guide/connect-deployed-zenml) to collaborate with your colleagues.
- Run the same pipeline on a [cloud MLOps stack in production](https://docs.zenml.io/user-guide/production-guide/cloud-stack).
- Track your metrics in an experiment tracker like [MLflow](https://docs.zenml.io/stacks-and-components/component-guide/experiment-trackers/mlflow).

## What next?

* If you have questions or feedback... join our [**Slack Community**](https://zenml.io/slack) and become part of the ZenML family!
* If you want to quickly get started with ZenML, check out the [ZenML Cloud](https://zenml.io/cloud).